<a href="https://colab.research.google.com/github/fjadidi2001/fake_news_detection/blob/main/BertGnn_Apr1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The BERT model was proposed in BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding by Jacob Devlin, Ming-Wei Chang, Kenton Lee and Kristina Toutanova. It’s a bidirectional transformer pretrained using a combination of masked language modeling objective and next sentence prediction on a large corpus comprising the Toronto Book Corpus and Wikipedia.



In [2]:
import pandas as pd
import numpy as np
import torch
from transformers import BertTokenizer, BertModel
from tensorflow.keras import layers, Model
import tensorflow as tf
from google.colab import drive
drive.mount('/content/drive/')
# Load and preprocess the data
df = pd.read_csv('/content/drive/MyDrive/Projects/Hayat/facebook-fact-check.csv', encoding='latin-1')

# Text preprocessing for BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
max_len = 100

def encode_texts(texts):
    return tokenizer(
        texts.tolist(),
        max_length=max_len,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )

text_encodings = encode_texts(df['Context Post'].fillna(''))
input_ids = text_encodings['input_ids']
attention_mask = text_encodings['attention_mask']

# Social features preprocessing
social_features = ['share_count', 'reaction_count', 'comment_count']
social_data = df[social_features].fillna(0).values
social_data = (social_data - social_data.mean()) / (social_data.std() + 1e-7)

# Labels for fake news detection
rating_map = {
    'no factual content': 0,
    'mostly true': 1,
    # Add more ratings based on your dataset
    # For fake news: 0 = fake/not factual, 1 = true/factual
}
labels = df['Rating'].map(rating_map).fillna(0).values

# Convert torch tensors to tf tensors
input_ids_tf = tf.convert_to_tensor(input_ids.numpy())
attention_mask_tf = tf.convert_to_tensor(attention_mask.numpy())
social_data_tf = tf.convert_to_tensor(social_data)
labels_tf = tf.convert_to_tensor(labels)

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [6]:
import pandas as pd
import numpy as np
from transformers import BertTokenizer, TFBertModel
from tensorflow.keras import layers, Model
import tensorflow as tf

# Load and preprocess the data
df = pd.read_csv('/content/drive/MyDrive/Projects/Hayat/facebook-fact-check.csv', encoding='latin-1')

# Text preprocessing for BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
max_len = 100

def encode_texts(texts):
    return tokenizer(
        texts.tolist(),
        max_length=max_len,
        padding='max_length',
        truncation=True,
        return_tensors='tf'
    )

# Fixed: Changed 'ape_texts' to 'encode_texts'
text_encodings = encode_texts(df['Context Post'].fillna(''))
input_ids = text_encodings['input_ids']
attention_mask = text_encodings['attention_mask']

# Social features preprocessing
social_features = ['share_count', 'reaction_count', 'comment_count']
social_data = df[social_features].fillna(0).values
social_data = (social_data - social_data.mean()) / (social_data.std() + 1e-7)

# Labels
rating_map = {'no factual content': 0, 'mostly true': 1}
labels = df['Rating'].map(rating_map).fillna(0).values

# Convert to tf tensors
input_ids_tf = input_ids
attention_mask_tf = attention_mask
social_data_tf = tf.convert_to_tensor(social_data)
labels_tf = tf.convert_to_tensor(labels)

# Text Branch with TFBertModel
def create_text_branch(use_cnn=True):
    input_ids = layers.Input(shape=(max_len,), dtype=tf.int32)
    attention_mask = layers.Input(shape=(max_len,), dtype=tf.int32)

    bert = TFBertModel.from_pretrained('bert-base-uncased')
    bert_output = bert(input_ids, attention_mask=attention_mask)[0]

    if use_cnn:
        x = layers.Conv1D(64, 5, activation='relu')(bert_output)
        x = layers.MaxPooling1D(5)(x)

    x = layers.Flatten()(x)
    return [input_ids, attention_mask], x

# Social Branch
def create_social_branch(input_shape, use_cnn=True):
    inputs = layers.Input(shape=input_shape)
    x = layers.Dense(64, activation='relu')(inputs)

    if use_cnn:
        x = layers.Reshape((-1, 1))(x)
        x = layers.Conv1D(32, 3, activation='relu')(x)
        x = layers.MaxPooling1D(2)(x)

    x = layers.Flatten()(x)
    return inputs, x

# Complete model
def create_model():
    text_inputs, text_output = create_text_branch(use_cnn=True)
    social_input, social_output = create_social_branch((len(social_features),), use_cnn=True)

    combined = layers.concatenate([text_output, social_output])
    x = layers.Dense(128, activation='relu')(combined)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)

    model = Model(inputs=[text_inputs, social_input], outputs=outputs)
    return model

# Create and compile
model = create_model()
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
)

model.summary()

# Train
history = model.fit(
    [input_ids_tf, attention_mask_tf, social_data_tf],
    labels_tf,
    epochs=10,
    batch_size=16,
    validation_split=0.2
)

# Evaluation function
def evaluate_fake_news(model, input_data, true_labels):
    predictions = model.predict(input_data)
    pred_binary = (predictions > 0.5).astype(int)
    accuracy = np.mean(pred_binary == true_labels)
    print(f"Accuracy: {accuracy:.4f}")
    return predictions

predictions = evaluate_fake_news(
    model,
    [input_ids_tf, attention_mask_tf, social_data_tf],
    labels
)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

ValueError: Exception encountered when calling layer 'tf_bert_model' (type TFBertModel).

Data of type <class 'keras.src.backend.common.keras_tensor.KerasTensor'> is not allowed only (<class 'tensorflow.python.framework.tensor.Tensor'>, <class 'bool'>, <class 'int'>, <class 'transformers.utils.generic.ModelOutput'>, <class 'tuple'>, <class 'list'>, <class 'dict'>, <class 'numpy.ndarray'>) is accepted for attention_mask.

Call arguments received by layer 'tf_bert_model' (type TFBertModel):
  • input_ids=<KerasTensor shape=(None, 100), dtype=int32, sparse=False, name=keras_tensor_24>
  • attention_mask=<KerasTensor shape=(None, 100), dtype=int32, sparse=False, name=keras_tensor_25>
  • token_type_ids=None
  • position_ids=None
  • head_mask=None
  • inputs_embeds=None
  • encoder_hidden_states=None
  • encoder_attention_mask=None
  • past_key_values=None
  • use_cache=None
  • output_attentions=None
  • output_hidden_states=None
  • return_dict=None
  • training=False